In [6]:
from netCDF4 import Dataset
import numpy as np
import iris
import os
os.chdir("/home/548/eh6215/Desktop/python/BARPA_evaluation/emma/long_trials/ACS_consistent")
import spatial_selection
import geopandas as gp


In [3]:
nrm_clusters = gp.read_file('/short/tp28/eh6215/NRM_clusters/NRM_clusters.shp')
nrm_sub_clusters = gp.read_file('/short/tp28/eh6215//NRM_sub_clusters/NRM_sub_clusters.shp')

In [14]:
lsm = iris.load_cube("/g/data/tp28/dev/barpa/ancils/barpa-r/land_sea_mask/cci/qrparm.mask.nc","land_binary_mask")
lon= lsm.coord('longitude').points
lat =lsm.coord('latitude').points
lonbnd = lsm.coord('longitude').bounds
latbnd = lsm.coord('latitude').bounds

In [34]:
a = spatial_selection.centre_mask(nrm_sub_clusters, lon,lat, output="2D")
lbl_a=np.asarray([['Wet Tropics', 'Rangelands', 'Monsoonal North', 'Monsoonal North',
       'East Coast', 'Central Slopes', 'Murray Basin',
       'Southern and South Western Flatlands',
       'Southern and South Western Flatlands', 'Southern Slopes',
       'Southern Slopes', 'Southern Slopes', 'Southern Slopes',
       'East Coast', 'Rangelands']])

a = a.values
a = np.ma.masked_array(a,np.isnan(a))

a.mask += (lsm.data == 0)

In [38]:
b = spatial_selection.centre_mask(nrm_clusters, lon,lat, output="2D")
lbl_b=np.asarray(['CentralSlopes', 'EastCoast', 'MurrayBasin', 'MonsoonalNorth', 'Rangelands', 'Southern_Slopes', 'SSW_Flatlands', 'Wet_Tropics'])

b = b.values
b = np.ma.masked_array(b,np.isnan(b))
b.mask += (lsm.data == 0)

In [36]:
# Create netCDF dimensions
dset = Dataset("/home/548/eh6215/NRM_subclusters_new.nc",mode="w")
dset.createDimension("lat" ,size=lat.size)
dset.createDimension("lon" ,size=lon.size)
dset.createDimension("nb"  ,size=2       )
dset.createDimension("n"   ,size=lbl_a.size)

# Create netCDF variables
X  = dset.createVariable("lat"        ,lat.dtype,("lat"      ))
XB = dset.createVariable("lat_bounds" ,lat.dtype,("lat","nb" ))
Y  = dset.createVariable("lon"        ,lon.dtype,("lon"      ))
YB = dset.createVariable("lon_bounds" ,lon.dtype,("lon","nb" ))
I  = dset.createVariable("ids"        ,a.dtype,("lat","lon"))
L  = dset.createVariable("labels"     ,lbl_a.dtype,("n"        ))

# Load data and encode attributes
X [...] = lat
X.units = "degrees_north"
XB[...] = latbnd

Y [...] = lon
Y.units = "degrees_east"
YB[...] = lonbnd

I[...]  = a
I.labels= "labels"

L[...]  = lbl_a

dset.close()

In [39]:
# Create netCDF dimensions
dset = Dataset("/home/548/eh6215/NRM_clusters_new.nc",mode="w")
dset.createDimension("lat" ,size=lat.size)
dset.createDimension("lon" ,size=lon.size)
dset.createDimension("nb"  ,size=2       )
dset.createDimension("n"   ,size=lbl_b.size)

# Create netCDF variables
X  = dset.createVariable("lat"        ,lat.dtype,("lat"      ))
XB = dset.createVariable("lat_bounds" ,lat.dtype,("lat","nb" ))
Y  = dset.createVariable("lon"        ,lon.dtype,("lon"      ))
YB = dset.createVariable("lon_bounds" ,lon.dtype,("lon","nb" ))
I  = dset.createVariable("ids"        ,b.dtype,("lat","lon"))
L  = dset.createVariable("labels"     ,lbl_b.dtype,("n"        ))

# Load data and encode attributes
X [...] = lat
X.units = "degrees_north"
XB[...] = latbnd

Y [...] = lon
Y.units = "degrees_east"
YB[...] = lonbnd

I[...]  = b
I.labels= "labels"

L[...]  = lbl_b

dset.close()

In [33]:
b.values

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])